<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>

# Hyperparameter Tuning

## *Data Science Unit 4 Sprint 2 Assignment 4*

## Your Mission, should you choose to accept it...

To hyperparameter tune and extract every ounce of accuracy out of this telecom customer churn dataset: 
https://lambdaschool-data-science.s3.amazonaws.com/telco-churn/WA_Fn-UseC_-Telco-Customer-Churn+(1).csv

<https://drive.google.com/file/d/1dfbAsM9DwA7tYhInyflIpZnYs7VT-0AQ/view>  (old link)

## Requirements

- Load the data
- Clean the data if necessary (it will be)
- Create and fit a baseline Keras MLP model to the data.
- Hyperparameter tune (at least) the following parameters:
 - batch_size
 - training epochs
 - optimizer
 - learning rate (if applicable to optimizer)
 - momentum (if applicable to optimizer)
 - activation functions
 - network weight initialization
 - dropout regularization
 - number of neurons in the hidden layer
 
 You must use Grid Search and Cross Validation for your initial pass of the above hyperparameters
 
 Try and get the maximum accuracy possible out of this data! You'll save big telecoms millions! Doesn't that sound great?


In [22]:
##### Your Code Here #####
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy
# fix random seed for reproducibility
numpy.random.seed(45)
pd.set_option('display.max_columns', 500)

In [24]:
df = pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')
print(df.shape)
df.head()

(7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


In [29]:
samples = df.shape[0]

oe = OrdinalEncoder()
df_transformed = oe.fit_transform(df)
X_train, X_test, y_train, y_test = train_test_split(df_transformed[:, :-1], df_transformed[:, -1], 
                                                    test_size=0.2, random_state=45)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(5634, 20) (1409, 20) (5634,) (1409,)


### Baseline

In [39]:
# Important Hyperparameters
inputs = X_train.shape[1]
epochs = 10
batch_size = 64

# Create Model
model = Sequential()
model.add(Dense(64, activation='relu', input_shape=(inputs,)))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
# Compile Model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_21 (Dense)             (None, 64)                1344      
_________________________________________________________________
dense_22 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_23 (Dense)             (None, 1)                 65        
Total params: 5,569
Trainable params: 5,569
Non-trainable params: 0
_________________________________________________________________


In [40]:
%%time
baseline = model.fit(X_train, y_train, 
          validation_data=(X_test,y_test), 
          epochs=epochs, 
          batch_size=batch_size
         )
scores = model.evaluate(X_test, y_test, verbose=0)
print(f"{model.metrics_names[1]}: {scores[1]}")

Train on 5634 samples, validate on 1409 samples
Epoch 1/10
5634/5634 [==============================] - 1s 249us/sample - loss: 10.3834 - accuracy: 0.6129 - val_loss: 3.8424 - val_accuracy: 0.7424
Epoch 2/10
5634/5634 [==============================] - 0s 45us/sample - loss: 3.2330 - accuracy: 0.6521 - val_loss: 1.8543 - val_accuracy: 0.7260
Epoch 3/10
5634/5634 [==============================] - 0s 45us/sample - loss: 2.9235 - accuracy: 0.6651 - val_loss: 1.9371 - val_accuracy: 0.7268
Epoch 4/10
5634/5634 [==============================] - 0s 50us/sample - loss: 3.2690 - accuracy: 0.6747 - val_loss: 2.1597 - val_accuracy: 0.5912
Epoch 5/10
5634/5634 [==============================] - 0s 37us/sample - loss: 3.3442 - accuracy: 0.6885 - val_loss: 3.3824 - val_accuracy: 0.7473
Epoch 6/10
5634/5634 [==============================] - 0s 37us/sample - loss: 2.6351 - accuracy: 0.7015 - val_loss: 3.2516 - val_accuracy: 0.7523
Epoch 7/10
5634/5634 [==============================] - 0s 37us/samp

### Grid Search

In [42]:
inputs = X_train.shape[1]

def create_model():
    model = Sequential()
    model.add(Dense(64, activation='relu', input_shape=(inputs,)))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

# create model
model = KerasClassifier(build_fn=create_model, verbose=0)

# define the grid search parameters
# batch_size = [10, 20, 40, 60, 80, 100]
# param_grid = dict(batch_size=batch_size, epochs=epochs)

# define the grid search parameters
param_grid = {'batch_size': [10, 20, 40, 60, 80, 100],
              'epochs': [10]}

# Create Grid Search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
grid_result = grid.fit(X_train, y_train)

# Report Results
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f"Means: {mean}, Stdev: {stdev} with: {param}")

Best: 0.7492054462432861 using {'batch_size': 10, 'epochs': 10}
Means: 0.7492054462432861, Stdev: 0.02243739781158063 with: {'batch_size': 10, 'epochs': 10}
Means: 0.7009073138237, Stdev: 0.04499778146369103 with: {'batch_size': 20, 'epochs': 10}
Means: 0.7302113056182862, Stdev: 0.06569439581259381 with: {'batch_size': 40, 'epochs': 10}
Means: 0.7122939109802247, Stdev: 0.09318083011746654 with: {'batch_size': 60, 'epochs': 10}
Means: 0.6267355024814606, Stdev: 0.11861297110288908 with: {'batch_size': 80, 'epochs': 10}
Means: 0.6668596148490906, Stdev: 0.11601123417873554 with: {'batch_size': 100, 'epochs': 10}


## Stretch Goals:

- Try to implement Random Search Hyperparameter Tuning on this dataset
- Try to implement Bayesian Optimiation tuning on this dataset using hyperas or hyperopt (if you're brave)
- Practice hyperparameter tuning other datasets that we have looked at. How high can you get MNIST? Above 99%?
- Study for the Sprint Challenge
 - Can you implement both perceptron and MLP models from scratch with forward and backpropagation?
 - Can you implement both perceptron and MLP models in keras and tune their hyperparameters with cross validation?